In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import sys
sys.path.append("../")
from PythonCode.preprocess.preprocess import *
from PythonCode.preprocess.complexStyleFeatures import *
from PythonCode.preprocess.simpleStyleFeatures import *
import seaborn as sns
import gensim
from gensim.utils import simple_preprocess
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle 
import gensim.downloader
from pprint import pprint
DATA_PATH_TRAIN = "../Data/C50/C50train/"
DATA_PATH_TEST = "../Data/C50/C50test/"

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


# Simple

In [2]:
df_train = load_data("../Data/C50/C50train",50)
df_train

,author_name,book_name,book_text
0,AaronPressman,106247newsML.txt,The Internet may be overflowing with new techn...
1,AaronPressman,120600newsML.txt,The U.S. Postal Service announced Wednesday a ...
2,AaronPressman,120683newsML.txt,Elementary school students with access to the ...
3,AaronPressman,136958newsML.txt,An influential Internet organisation has backe...
4,AaronPressman,137498newsML.txt,An influential Internet organisation has backe...
...,...,...,...
2495,WilliamKazer,28223newsML.txt,China's central bank chief has said that infla...
2496,WilliamKazer,282935newsML.txt,"China ushered in 1997, a year it has hailed as..."
2497,WilliamKazer,287736newsML.txt,China issued tough new rules on the handling o...
2498,WilliamKazer,289747newsML.txt,China will avoid bold moves in tackling its ai...


In [5]:
df_test = load_data("../Data/C50/C50test",50)
df_test

,author_name,book_name,book_text
0,AaronPressman,421829newsML.txt,U.S. Senators on Tuesday sharply criticized a ...
1,AaronPressman,424074newsML.txt,Two members of Congress criticised the Federal...
2,AaronPressman,42764newsML.txt,Commuters stuck in traffic on the Leesburg Pik...
3,AaronPressman,43033newsML.txt,A broad coalition of corporations went to Capi...
4,AaronPressman,433558newsML.txt,"On the Internet, where new products come and g..."
...,...,...,...
2495,WilliamKazer,504283newsML.txt,China has scored new successes in its fight ag...
2496,WilliamKazer,504526newsML.txt,China has scored new successes in its fight ag...
2497,WilliamKazer,51502newsML.txt,China is on target with plans to to promote 10...
2498,WilliamKazer,522090newsML.txt,China may need to adjust the mix of its treasu...


In [7]:
df = df_train.append(df_test, ignore_index=True)

In [9]:
TEST_PART = 0.1

## Baseline

### Pre

In [10]:
X_train,X_test,y_train,y_test = train_test_split(df["book_text"],df["author_name"],test_size=TEST_PART)

In [20]:
def avg_word_len(df: pd.DataFrame) -> pd.DataFrame:
    avg_word_len = df.astype(str).swifter.apply(
        lambda s: pd.Series(nltk.word_tokenize(s)).map(len).mean()).rename("avg_word_len")
    return pd.DataFrame(avg_word_len)


def avg_sentence_len(df: pd.DataFrame) -> pd.DataFrame:
    sentence_count = df.astype(str).swifter.apply(
        lambda text: pd.Series(nltk.sent_tokenize(text)).map(
            lambda sent: len(nltk.word_tokenize(sent))).mean()).rename("avg_sentence_len")

    return pd.DataFrame(sentence_count)

def hapax_disLegemena(x: pd.DataFrame) -> pd.DataFrame:
    """
    hapax disLegemena Measure for lexicographic diversity: v2/N
    v2 = number of words that appears twice
    N = number of words or number of unique words
    """

    def get_hapax_disLegemena(text: str) -> (float, float):
        words = nltk.word_tokenize(text)
        words_count = Counter(words)
        v2 = len([_ for _, count in words_count.items() if count == 2])
        return pd.Series((v2 / (len(words) + EPSILON), v2 / (len(set(words))+EPSILON)))

    return pd.DataFrame(x.astype(str).swifter.apply(get_hapax_disLegemena)).set_axis(['hapax_disLegemena(H)', 'hapax_disLegemena(S)'], axis=1)


In [21]:
def preprocess(x_train: pd.DataFrame, x_test: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    def helper(x: pd.DataFrame) -> pd.DataFrame:
        return pd.concat([avg_word_len(x),avg_sentence_len(x),hapax_disLegemena(x)], axis=1)
        
    return helper(x_train), helper(x_test)

X_train,X_test = preprocess(X_train,X_test)

Pandas Apply:   0%|          | 0/4500 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4500 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/4500 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/500 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/500 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/500 [00:00<?, ?it/s]

In [23]:
y_train = pd.Categorical(y_train).codes
y_test = pd.Categorical(y_test).codes

### Model

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100,max_depth=7, random_state=0)
model.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=7, random_state=0)

In [25]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.17      0.40      0.24         5
           1       0.30      0.23      0.26        13
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00        10
           4       0.09      0.11      0.10         9
           5       0.00      0.00      0.00         8
           6       0.10      0.09      0.10        11
           7       0.16      0.23      0.19        13
           8       0.22      0.20      0.21        10
           9       0.12      0.12      0.12         8
          10       0.07      0.14      0.09         7
          11       0.00      0.00      0.00        10
          12       0.14      0.08      0.11        12
          13       0.06      0.20      0.09         5
          14       0.00      0.00      0.00         9
          15       0.38      0.23      0.29        13
          16       0.10      0.07      0.08        14
          17       0.08    

# Using Glove

In [13]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [27]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [28]:
len(glove_vectors)

400000

### Sentence Level

In [36]:
X_train,X_test,y_train,y_test = train_test_split(df["book_text"],df["author_name"],test_size=TEST_PART)

In [43]:
def num_sentences_based_chucking(df: pd.DataFrame, chunk_size: int):
    rows = []
    for row in df:
        sentences = nltk.tokenize.sent_tokenize(row)
        curr_chunk = []
        for sentence in sentences:
            curr_chunk.append(sentence)
            if len(curr_chunk) == chunk_size:
                rows.append("".join(curr_chunk))
                curr_chunk = []
        rows.append("".join(curr_chunk))  # add the last one
    return pd.DataFrame(rows)

In [44]:
NUM_OF_SENTENCE_CHUNK = 3

In [45]:
X_train = num_sentences_based_chucking(X_train,NUM_OF_SENTENCE_CHUNK)
X_test = num_sentences_based_chucking(X_test,NUM_OF_SENTENCE_CHUNK)

In [51]:
X_train = X_train[0]
X_test = X_test[0]

In [52]:
X_train

0        China unveiled on Tuesday the country's first ...
1        The exchange, which has the direct support fro...
2        "With China's fast-growing economy and Shangha...
3        But the focus of the business nationwide will ...
4        Members of the exchange include ship-owners, m...
                               ...                        
35521    Chinese sources have said Wang faces a sentenc...
35522    Wang appears likely to be the next victim of a...
35523    Hopes that the award of the 1996 Nobel Peace P...
35524    The court's verdict said Wei, who was jailed f...
35525                                                     
Name: 0, Length: 35526, dtype: object

In [ ]:
def preprocess():
    pass
    # tranform each word by glove table(handle not found word as 'magic-word' and
    # delete number and special charcter before(trimming))

### Article Level

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df["book_text"],df["author_name"],test_size=TEST_PART)